In [35]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import pandas as pd

model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl").to("cuda")
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")

KeyboardInterrupt: 

In [ ]:
def preprocess(dataset):
    dataset.rename(columns={'sentiment': 'Sentiment', 'review': 'Text'}, inplace=True)
    dataset['Sentiment'] = dataset['Sentiment'].replace({'negative': 0, 'positive': 1})
    dataset['Sentiment'] = dataset['Sentiment'].replace({'Negative': 0, 'Positive': 1})
    dataset = dataset.sample(frac=1, random_state=1).reset_index(drop=True)


In [ ]:
original = pd.read_csv("data/test_original.tsv", sep='\t')
preprocess(original)

/tmp/ipykernel_66631/2921636663.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset['Sentiment'] = dataset['Sentiment'].replace({'Negative': 0, 'Positive': 1})


In [ ]:
contrast = pd.read_csv("data/test_contrast.tsv", sep='\t')
preprocess(contrast)

/tmp/ipykernel_66631/2921636663.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset['Sentiment'] = dataset['Sentiment'].replace({'Negative': 0, 'Positive': 1})


In [ ]:
def inference(*, prompt, sentence, tokenizer, model):
    inputs = tokenizer(prompt(sentence),  padding='max_length', return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=50)
    result = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    
    if "negative" in result.lower():
        return 0
    elif "positive" in result.lower():
        return 1
    else:
        return 2

In [ ]:
import random


def build_example(dataset, index):
    return ("Text: " + dataset.Text[index] + "\nSentiment: " + ["negative","positive"][dataset.Sentiment[index]] + "\n\n")

def get_build_prompt(dataset, index, shots, system):
    def build_prompt(sent):
        ret = ""
        if(system):
            if(shots > 0):
                ret += "Classify the text into negative or positive. Here you have some examples:\n\n"
            else:
                ret += "Classify the text into negative or positive.\n\n"
        help_list = []
        for j in range(shots):
            help = random.sample(range(len(dataset)), 1)[0]
            while(index == help or help in help_list):
                help = random.sample(range(len(dataset)), 1)[0]
            help_list.append(help)
            example = build_example(dataset, help)
            ret += example
        ret += "Text: " + sent + "\nSentiment: "
        return ret
    return build_prompt

example = get_build_prompt(original, 1, shots=4, system=False)
print(example("My Sentence"))


Text: A wonderful early musical film from Rene Clair, as fun and witty as his silent "The Italian Straw Hat". Using sound in a expressive way and not just for dialogue and effects, Clair influenced early musicals in America (the opera scene from A Night at the Opera is strongly influenced by Le Million, for example). Should (but won't) be seen by all cinephiles, and the DVD from Criterion is exactly as good as you'd expect. There's not a ton of extras, but most DVD extras I've seen are useless fluff, and the Clair interview on disc is one I hadn't ever seen. Get it while it's still around.
Sentiment: positive

Text: Melissa's sixteenth birthday is right around the corner and she's just discovering her sexuality with boys. But it turns out that all the guys that she spends time with all wind up murdered in this generic '80's slasher film. It's up to the local town sheriff Dan Burke (Bo Hopkins, The Wild Bunch) and his annoying mystery-loving goody two-shoes daughter, Marci (Dana Kimmell

In [ ]:
import random
from sklearn.metrics import classification_report
import torch
from tqdm import tqdm


@torch.no_grad()
def eval(dataset, system):
    overall_results = dict()

    for shots in [0,1,2,3]:
        if(shots == 0):
             test = 1
        else:
            test = 3
        print("Running shot", shots)
        for run in range(test):
            print("Run", run)
            targets = []
            predictions = []
            total = len(dataset)
            
            for i in tqdm(range(total), total=total):
                prompt = get_build_prompt(dataset, i, shots, system)
                targets.append(dataset.Sentiment[i])
                predictions.append(inference(prompt=prompt, sentence=dataset.Text[i], tokenizer=tokenizer, model=model))
            
            right, wrong, no_sentiment = 0, 0, 0
            
            for j in range(len(predictions)):
                if(predictions[j] == 2):
                    no_sentiment += 1
                elif(targets[j] == predictions[j]):
                        right += 1
                elif(targets[j] != predictions[j]):
                        wrong += 1

            if shots not in overall_results:
                overall_results[shots] = dict()

            overall_results[shots][run] = {'correct' : right/len(predictions), 'wrong' : wrong/len(predictions), 'no_sentiment': no_sentiment/len(predictions)}
            # report = classification_report(targets, predictions, labels=[0,1,2], output_dict=True, zero_division=0)
            # overall_results[shots] = report

    return overall_results

In [ ]:
import json
import csv

def write_csv(file_path, dataframe):
    csv_path = "csv/" + file_path + ".csv"
    json_path = "json/" + file_path + ".json"
    with open(csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # Scrittura dell'intestazione
        writer.writerow(['', '', file_path.replace("_results", ""), ''])
        writer.writerow(['', '', 'Errors', '', '', 'Errors', '', '', 'Errors', '', '', 'Errors'])
        writer.writerow(['Runs', 'Correct', 'Wrong', 'No_sentiment', 'Correct', 'Wrong', 'No_sentiment', 'Correct', 'Wrong', 'No_sentiment', 'Correct', 'Wrong', 'No_sentiment'])
        
        # Scrittura dei dati
        for j in range(3):
            field = ["Run " + str(j)]
            if(j != 0):
                field.append("")
                field.append("")
                field.append("")
            for shots, runs in dataframe.items():
                for run_number, metrics in runs.items():
                    if(run_number == j):
                        field.append(metrics['correct'])
                        field.append(metrics['wrong'])
                        field.append(metrics['no_sentiment'])
            writer.writerow(field)
    print(f"I dati sono stati scritti su {csv_path}.")

    with open(json_path, 'w') as f:
        json.dump(dataframe, f, sort_keys=True, indent=4)

In [ ]:
test = eval(original, system=False)

file_name = "seed_test"

write_csv(file_name, test)

Running shot 0
Run 0


100%|██████████| 488/488 [01:12<00:00,  6.70it/s]


Running shot 1
Run 0


100%|██████████| 488/488 [01:16<00:00,  6.35it/s]


Run 1


...

Run 1


100%|██████████| 488/488 [02:12<00:00,  3.67it/s]


Run 2


100%|██████████| 488/488 [02:14<00:00,  3.64it/s]

I dati sono stati scritti su csv/seed_test.csv.


In [ ]:
averages = {}

for shot, runs in test.items():
    correct_total = 0
    no_sentiment_total = 0
    wrong_total = 0
    run_count = len(runs)
    
    for run_id, values in runs.items():
        correct_total += values['correct']
        no_sentiment_total += values['no_sentiment']
        wrong_total += values['wrong']
    
    averages[shot] = {
        'correct': correct_total / run_count,
        'no_sentiment': no_sentiment_total / run_count,
        'wrong': wrong_total / run_count
    }

# Creazione di un dataframe per visualizzare i risultati
averages_df = pd.DataFrame.from_dict(averages, orient='index')
print(averages_df)

    correct  no_sentiment     wrong
0  0.954918      0.008197  0.036885
1  0.960383      0.000000  0.039617
2  0.963115      0.000000  0.036885
3  0.963115      0.000000  0.036885


In [ ]:
accuracies = []

for shot, runs in test.items():
    for run_id, values in runs.items():
        accuracies.append({
            'shot': shot,
            'run': run_id,
            'correct': values['correct']
        })

# Creazione di un DataFrame per visualizzare i risultati
accuracies_df = pd.DataFrame(accuracies)
print(accuracies_df)

   shot  run   correct
0     0    0  0.954918
1     1    0  0.963115
2     1    1  0.959016
3     1    2  0.959016
4     2    0  0.963115
5     2    1  0.965164
6     2    2  0.961066
7     3    0  0.961066
8     3    1  0.963115
9     3    2  0.965164


In [37]:
accuracies = []

for shot, runs in test.items():
    for run_id, values in runs.items():
        accuracies.append({
            'shot': int(shot),
            'run': int(run_id),
            'correct': values['correct']
        })

# Creazione di un DataFrame
accuracies_df = pd.DataFrame(accuracies)

# Creare una griglia completa di combinazioni di shot e run
all_shots = accuracies_df['shot'].unique()
all_runs = accuracies_df['run'].unique()
complete_index = pd.MultiIndex.from_product([all_shots, all_runs], names=['shot', 'run'])

# Riorganizzare il DataFrame per avere dati completi con NaN dove mancano i dati
complete_accuracies_df = accuracies_df.set_index(['shot', 'run']).reindex(complete_index).reset_index()

# Plotting
plt.figure(figsize=(12, 6))

# Definizione della larghezza delle barre
bar_width = 0.2

# Generazione delle posizioni sull'asse x per ciascuna run
x_pos = np.arange(len(all_shots))

# Plot di ogni run
for i, run in enumerate(all_runs):
    run_data = complete_accuracies_df[complete_accuracies_df['run'] == run]
    bars = plt.bar(x_pos + i * bar_width, run_data['correct'], width=bar_width, label=f'Run {run}')
    
    # Aggiunta di etichette per i valori sopra ogni barra
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval + 0.0005, f'{yval:.3f}', ha='center', va='bottom', fontsize=10)

# Configurazione degli assi e delle etichette
plt.xlabel('Shot')
plt.ylabel('Correct Accuracy')
plt.title('Accuracy per Shot and Run')
plt.xticks(x_pos + bar_width * (len(all_runs) - 1) / 2, [f'Shot {shot}' for shot in all_shots])
plt.ylim(0.94, 0.97)  # Zoom sull'intervallo di valori di interesse
plt.legend(title='Runs')
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.show()


NameError: name 'text' is not defined

In [ ]:
import os
os.system("paplay complete.wav")


sh: 1: paplay: not found


32512